In [1]:
# base_pth = "/home/guangda/repos/LabelMaker-Pointcept/exp/scannet200/alc_pretrain_scannet200_ft/model/model_best.pth"
# ppt_pth = "/home/guangda/repos/LabelMaker-Pointcept/exp/scannet200/scannet200_s3dis_structure3d_alc_joint_training/model/model_best.pth"

ppt_pth = "/home/guangda/repos/LabelMaker-Pointcept/exp/scannet/scannet_s3dis_structure3d_alc_joint_training/model/model_best.pth"

In [2]:
import torch

In [3]:
a = torch.load(ppt_pth)
# b = torch.load(ppt_pth)

In [4]:
import os
import sys

sys.path.append("/home/guangda/repos/LabelMaker-Pointcept")

In [5]:
from pointcept.models import  PointTransformerV3
from pointcept.models.default import DefaultSegmentorV2
from pointcept.models.losses import CrossEntropyLoss, LovaszLoss

In [6]:
empty_model = DefaultSegmentorV2(
    num_classes=20,
    backbone_out_channels=64,
    backbone=dict(
        type="PT-v3m1",
        in_channels=6,
        order=("z", "z-trans", "hilbert", "hilbert-trans"),
        stride=(2, 2, 2, 2),
        enc_depths=(3, 3, 3, 6, 3),
        enc_channels=(48, 96, 192, 384, 512),
        enc_num_head=(3, 6, 12, 24, 32),
        enc_patch_size=(1024, 1024, 1024, 1024, 1024),
        dec_depths=(3, 3, 3, 3),
        dec_channels=(64, 96, 192, 384),
        dec_num_head=(4, 6, 12, 24),
        dec_patch_size=(1024, 1024, 1024, 1024),
        mlp_ratio=4,
        qkv_bias=True,
        qk_scale=None,
        attn_drop=0.0,
        proj_drop=0.0,
        drop_path=0.3,
        shuffle_orders=True,
        pre_norm=True,
        enable_rpe=False,
        enable_flash=True,
        upcast_attention=False,
        upcast_softmax=False,
        cls_mode=False,
        pdnorm_bn=True,
        pdnorm_ln=True,
        pdnorm_decouple=True,
        pdnorm_adaptive=False,
        pdnorm_affine=True,
        pdnorm_conditions=("ScanNet", "S3DIS", "Structured3D", "ALC", "ScanNet200"),
    ),
    criteria=[
        dict(type="CrossEntropyLoss", loss_weight=1.0, ignore_index=-1),
        dict(type="LovaszLoss", mode="multiclass", loss_weight=1.0, ignore_index=-1),
    ],
    freeze_backbone=True,
)

In [7]:
old_keys = set([key.replace("module.", "") for key in a["state_dict"].keys()])
new_keys = set(empty_model.state_dict().keys())

In [8]:
old_keys - new_keys

{'class_embedding',
 'embedding_table.weight',
 'logit_scale',
 'proj_head.bias',
 'proj_head.weight'}

In [9]:
new_keys - old_keys

{'seg_head.bias', 'seg_head.weight'}

In [10]:
new_save = {"state_dict": {}}

for key in a["state_dict"].keys():
    if key.replace("module.", "") in empty_model.state_dict().keys():
        print("1", key.replace("module.", ""))
        new_save["state_dict"]["module." + key.replace("module.", "")] = a["state_dict"][key]

for key in empty_model.state_dict().keys():
    if "module." + key not in new_save["state_dict"].keys():
        print("2", key)
        new_save["state_dict"]["module." + key] = empty_model.state_dict()[key]

1 backbone.embedding.stem.conv.weight
1 backbone.embedding.stem.norm.norm.0.weight
1 backbone.embedding.stem.norm.norm.0.bias
1 backbone.embedding.stem.norm.norm.0.running_mean
1 backbone.embedding.stem.norm.norm.0.running_var
1 backbone.embedding.stem.norm.norm.0.num_batches_tracked
1 backbone.embedding.stem.norm.norm.1.weight
1 backbone.embedding.stem.norm.norm.1.bias
1 backbone.embedding.stem.norm.norm.1.running_mean
1 backbone.embedding.stem.norm.norm.1.running_var
1 backbone.embedding.stem.norm.norm.1.num_batches_tracked
1 backbone.embedding.stem.norm.norm.2.weight
1 backbone.embedding.stem.norm.norm.2.bias
1 backbone.embedding.stem.norm.norm.2.running_mean
1 backbone.embedding.stem.norm.norm.2.running_var
1 backbone.embedding.stem.norm.norm.2.num_batches_tracked
1 backbone.embedding.stem.norm.norm.3.weight
1 backbone.embedding.stem.norm.norm.3.bias
1 backbone.embedding.stem.norm.norm.3.running_mean
1 backbone.embedding.stem.norm.norm.3.running_var
1 backbone.embedding.stem.norm.n

In [12]:
torch.save(new_save, "/home/guangda/repos/LabelMaker-Pointcept/exp/scannet/scannet_s3dis_structure3d_alc_joint_training/model/model_modified.pth")

In [13]:
load_save = torch.load("/home/guangda/repos/LabelMaker-Pointcept/exp/scannet200/scannet200_s3dis_structure3d_alc_joint_training/model/model_modified.pth")

In [14]:
load_save["state_dict"].keys()

dict_keys(['module.backbone.embedding.stem.conv.weight', 'module.backbone.embedding.stem.norm.norm.0.weight', 'module.backbone.embedding.stem.norm.norm.0.bias', 'module.backbone.embedding.stem.norm.norm.0.running_mean', 'module.backbone.embedding.stem.norm.norm.0.running_var', 'module.backbone.embedding.stem.norm.norm.0.num_batches_tracked', 'module.backbone.embedding.stem.norm.norm.1.weight', 'module.backbone.embedding.stem.norm.norm.1.bias', 'module.backbone.embedding.stem.norm.norm.1.running_mean', 'module.backbone.embedding.stem.norm.norm.1.running_var', 'module.backbone.embedding.stem.norm.norm.1.num_batches_tracked', 'module.backbone.embedding.stem.norm.norm.2.weight', 'module.backbone.embedding.stem.norm.norm.2.bias', 'module.backbone.embedding.stem.norm.norm.2.running_mean', 'module.backbone.embedding.stem.norm.norm.2.running_var', 'module.backbone.embedding.stem.norm.norm.2.num_batches_tracked', 'module.backbone.embedding.stem.norm.norm.3.weight', 'module.backbone.embedding.st